In [1]:
%%bash

pip install --upgrade pip
pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab,ocr]
pip install farm-haystack[opensearch]

# For Colab/linux based machines:
wget https://dl.xpdfreader.com/xpdf-tools-linux-4.04.tar.gz
tar -xvf xpdf-tools-linux-4.04.tar.gz && sudo cp xpdf-tools-linux-4.04/bin64/pdftotext /usr/local/bin


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 25.1 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-install-6q4mcunq/farm-haystack_8a68c71099904f4c9b089e4d19642a39
  Resolved https://github.com/deepset-ai/haystack.git to commit b9ab7b3ca2d548b3669038d837b83e6558fa8c6d
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ━━━━━━━━━━━

DEPRECATION: git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab,ocr] contains an egg fragment with a non-PEP 508 name pip 25.0 will enforce this behaviour change. A possible replacement is to use the req @ url syntax, and remove the egg fragment. Discussion can be found at https://github.com/pypa/pip/issues/11617
  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-install-6q4mcunq/farm-haystack_8a68c71099904f4c9b089e4d19642a39
--2023-02-05 14:09:07--  https://dl.xpdfreader.com/xpdf-tools-linux-4.04.tar.gz
Resolving dl.xpdfreader.com (dl.xpdfreader.com)... 45.79.72.155
Connecting to dl.xpdfreader.com (dl.xpdfreader.com)|45.79.72.155|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23687259 (23M) [application/x-gzip]
Saving to: ‘xpdf-tools-linux-4.04.tar.gz’

     0K .......... .......... .......... .......... ..........  0%  149K 2m35s
    50K .......... .......... .......... .......... .

In [ ]:

'''
import boto3
s3 = boto3.client("s3",
                  aws_access_key_id='your_access_key',
                  aws_secret_access_key='your_secret_key')


# Connect to the S3 service 
s3 = boto3.client("s3")

def upload_to_aws(local_file, s3_file):
    s3.upload_file(local_file, "your-bucket-name", s3_file)
    print("Upload Successful")
    return True

# Call the upload function
uploaded = upload_to_aws("local-file-path", "s3-file-path")
'''

#CREATE A FOLDER FOR DOCUMENTS
Place all documents in a folder, and copy the path of the folder doc_dir="your_folder_name"


In [19]:
doc_dir = "/content/data_user2"
user_id_inserted_into_meta = 'user2' #first user is "12345XYZ"

INFO:haystack.utils.preprocessing:Converting /content/data_user2/Slides.docx


In [35]:

def load_directory(directory, username): 

    doc_dir = directory #"/content/data_user2"
    user_id_inserted_into_meta = username #'user2' #first user is "12345XYZ"

    import logging
    logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
    logging.getLogger("haystack").setLevel(logging.INFO)

    #converting files from doc directory
    from haystack.utils import convert_files_to_docs
    all_docs = convert_files_to_docs(dir_path=doc_dir)

    #preprocessor setup
    from haystack.nodes import PreProcessor

    all_docs = convert_files_to_docs(dir_path=doc_dir)

    preprocessor = PreProcessor(
        clean_empty_lines=True,
        clean_whitespace=True,
        clean_header_footer=False,
        split_by="word",
        split_length=50,
        split_overlap= 6,
        split_respect_sentence_boundary=True,
    )
    snippets = preprocessor.process(all_docs)
    for snippet in snippets:
        snippet.meta["user_id"] = user_id_inserted_into_meta


    print(f"n_files_input: {len(all_docs)}\nn_docs_output: {len(snippets)}")
#print(type(snippets))


In [ ]:
print(type(snippets[0]))

<class 'haystack.schema.Document'>


#embedd the documents
Connect to the OpenSearch Database 
Then embedd each snippet

In [44]:
def write_to_database(database_username, database_password):

    import urllib3
    urllib3.disable_warnings()


    from haystack.document_stores import OpenSearchDocumentStore
    from opensearchpy import OpenSearch, Urllib3HttpConnection, RequestsHttpConnection, NotFoundError, RequestError
    from opensearchpy.helpers import bulk, scan


    document_store = OpenSearchDocumentStore(
        similarity="dot_product",
        embedding_dim=128,
        host='search-dlopensearchlfqa2-h2m5uuvjfsjfjruwrucopcgagq.us-east-1.es.amazonaws.com',
        username= database_username, #'admin',
        password= database_password, #'Admin12345?',
        port=443
    )

    ### Initialize the Retriever

    import urllib3
    urllib3.disable_warnings()


    from haystack.nodes import DensePassageRetriever

    retriever = DensePassageRetriever(
        document_store=document_store,
        query_embedding_model="vblagoje/dpr-question_encoder-single-lfqa-wiki",
        passage_embedding_model="vblagoje/dpr-ctx_encoder-single-lfqa-wiki",
    )

    document_store.write_documents(snippets, duplicate_documents = "skip") 

    document_store.update_embeddings(retriever, update_existing_embeddings=False)



In [ ]:
####### FOR UPDATING THE DATABASE ########

##### DO NOT USE UNLESS YOU WANT TO WAIT 1+ Hrs ######







#Answer Questions From a Query

In [ ]:
#enter in the query
'''
query_="would the discarge of debt be income?"

from haystack.utils import print_documents
from haystack.pipelines import DocumentSearchPipeline

p_retrieval = DocumentSearchPipeline(retriever)
res = p_retrieval.run(query=query_, params={"Retriever": {"top_k": 10}})
#print_documents(res, max_text_len=512)

#reader and Answer Genereator


from haystack.nodes import Seq2SeqGenerator
generator = Seq2SeqGenerator(model_name_or_path="vblagoje/bart_lfqa")

from haystack.pipelines import GenerativeQAPipeline
pipe = GenerativeQAPipeline(generator, retriever)

'''

In [31]:
def run_query(user_query,retriever_number,user_id):
      from sqlalchemy.orm import query_expression
      from haystack.utils import print_documents
      from haystack.pipelines import DocumentSearchPipeline

      from haystack.nodes import Seq2SeqGenerator
      generator = Seq2SeqGenerator(model_name_or_path="vblagoje/bart_lfqa")

      from haystack.pipelines import GenerativeQAPipeline
      pipe = GenerativeQAPipeline(generator, retriever)


      print(user_id_inserted_into_meta)
      user_id_of_user = user_id


      query_use=user_query #"example: What is a Side Letter Offering Document used for?"

      p_retrieval = DocumentSearchPipeline(retriever)
      results = p_retrieval.run(query=query_use, params={"Retriever": {"top_k": retriever_number},
                                                      "filters": {"user_id": [user_id_of_user]}
                                                      })

      from pprint import pprint
      pprint(pipe.run(
          query=query_use, 
          params={"Retriever": {"top_k": 10},
                  "filters": {"user_id": [user_id_of_user]}}
      ))

      print_documents(results, max_text_len=512)

In [ ]:
load_directory(directory="/content/data_user2",
               username="user2")
write_to_database(database_username='admin', database_password= 'Admin12345?')

In [46]:
run_query(user_query="What is a Side letter used for?",
          retriever_number=10,
          user_id="user2")

INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.schema:Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.
INFO:haystack.schema:Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.


user2


INFO:haystack.schema:Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.
INFO:haystack.schema:Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.
INFO:haystack.schema:Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.
INFO:haystack.schema:Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.
INFO:haystack.schema:Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.
INFO:haystack.schema:Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.
INFO:haystack.schema:Setting the ID manu

{'answers': [<Answer {'answer': "A side letter is an agreement between the fund and the limited partners of the fund. For example, if you invest in a private equity fund, you can enter into a side letter with the fund's limited partners in exchange for a reduction in fees.", 'type': 'generative', 'score': None, 'context': None, 'offsets_in_document': None, 'offsets_in_context': None, 'document_id': None, 'meta': {'doc_ids': ['d2f1cb881ae1c3455b97755b79868d5e', '1d5789b8ebbf4ad4cf8f9a42ec489a45', '3e5f9e5be45a54653d37bb32d1257f3a', 'e56f2c26262e66cb48e2a158d8a65855', '8fa880fe9069b0d54ba0d595ee5fc015', '625d78f69b68ff51630fb25a20e04814', 'd6c92720af7b4116d989564a2c70a61e', 'daa30f28edf1aa0652c27abf806ce077', '4fd594963c99c54c4478b4ae7cd107e6', '4e35558030b488a54590598db327cea0'], 'doc_scores': [0.5322329166750075, 0.5301753647211532, 0.5293185534139392, 0.528812240187411, 0.5283475509503403, 0.5279295889153314, 0.5278793832765006, 0.5277860234641786, 0.5260714579587046, 0.52572915145568